In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
dataframe = pd.read_csv("train_indessa.csv")
dataframe =dataframe.drop(['member_id','emp_title','zip_code','addr_state','dti','batch_enrolled','sub_grade','desc','title','verification_status_joint'],axis=1)
dataframe = dataframe[0:10]

In [3]:
dataframe

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,grade,emp_length,home_ownership,annual_inc,verification_status,...,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status
0,14350,14350,14350.0,36 months,19.19,E,9 years,OWN,28700.0,Source Verified,...,0.0,0.0,74.0,INDIVIDUAL,26th week,0.0,0.0,28699.0,30800.0,0
1,4800,4800,4800.0,36 months,10.99,B,< 1 year,MORTGAGE,65000.0,Source Verified,...,0.0,0.0,NaN,INDIVIDUAL,9th week,0.0,0.0,9974.0,32900.0,0
2,10000,10000,10000.0,36 months,7.26,A,2 years,OWN,45000.0,Not Verified,...,0.0,0.0,NaN,INDIVIDUAL,9th week,0.0,65.0,38295.0,34900.0,0
3,15000,15000,15000.0,36 months,19.72,D,10+ years,RENT,105000.0,Not Verified,...,0.0,0.0,NaN,INDIVIDUAL,135th week,0.0,0.0,55564.0,24700.0,0
4,16000,16000,16000.0,36 months,10.64,B,10+ years,RENT,52000.0,Verified,...,0.0,0.0,NaN,INDIVIDUAL,96th week,0.0,0.0,47159.0,47033.0,0
5,15000,15000,14950.0,36 months,8.90,A,2 years,MORTGAGE,120000.0,Not Verified,...,0.0,0.0,NaN,INDIVIDUAL,113th week,0.0,0.0,350619.0,29500.0,0
6,5000,5000,4975.0,36 months,7.90,A,5 years,RENT,75000.0,Source Verified,...,0.0,0.0,NaN,INDIVIDUAL,117th week,0.0,1023.0,13272.0,55500.0,1
7,6000,6000,6000.0,36 months,9.17,B,8 years,MORTGAGE,54000.0,Not Verified,...,0.0,0.0,54.0,INDIVIDUAL,78th week,0.0,0.0,272579.0,11800.0,0
8,6000,6000,6000.0,36 months,13.99,C,7 years,MORTGAGE,92000.0,Not Verified,...,0.0,0.0,NaN,INDIVIDUAL,44th week,0.0,0.0,281521.0,62100.0,0
9,34550,34550,34550.0,60 months,17.14,D,2 years,MORTGAGE,72000.0,Verified,...,0.0,0.0,NaN,INDIVIDUAL,52th week,0.0,0.0,76034.0,33200.0,0


In [4]:
dataframe.loc[:,('nstatus')] = dataframe['loan_status'] == 0
dataframe.loc[:,('nstatus')] = dataframe['nstatus'].astype(int)
dataframe.loc[:,('term')] = dataframe['term'].str.split('months').str[0]
dataframe.loc[dataframe.grade == 'A'  ,('grade')] = '1' 
dataframe.loc[dataframe.grade == 'B'  ,('grade')] = '2' 
dataframe.loc[dataframe.grade == 'C'  ,('grade')] = '3' 
dataframe.loc[dataframe.grade == 'D'  ,('grade')] = '4' 
dataframe.loc[dataframe.grade == 'E'  ,('grade')] = '5' 
dataframe.loc[:,('emp_length')] = dataframe['emp_length'].str.split('years').str[0]
dataframe.loc[dataframe.emp_length == "< 1 year"  ,('emp_length')] = '0' 
dataframe.loc[dataframe.emp_length == "10+ "  ,('emp_length')] = '11' 
dataframe.loc[dataframe.home_ownership == "OWN"  ,('home_ownership')] = '1' 
dataframe.loc[dataframe.home_ownership == "RENT"  ,('home_ownership')] = '2' 
dataframe.loc[dataframe.home_ownership == "MORTGAGE"  ,('home_ownership')] = '3'
dataframe.loc[dataframe.verification_status == "Source Verified"  ,('verification_status')] = '1'
dataframe.loc[dataframe.verification_status == "Verified"  ,('verification_status')] = '2'
dataframe.loc[dataframe.verification_status == "Not Verified"  ,('verification_status')] = '3'
dataframe.loc[dataframe.mths_since_last_major_derog.isnull()  ,('mths_since_last_major_derog')] = '0'
dataframe.loc[dataframe.pymnt_plan == 'n'  ,('pymnt_plan')] = '1'
dataframe.loc[dataframe.purpose == 'debt_consolidation'  ,('purpose')] = '3'
dataframe.loc[dataframe.purpose == 'home_improvement'  ,('purpose')] = '2'
dataframe.loc[dataframe.purpose == 'credit_card'  ,('purpose')] = '3'
dataframe.loc[dataframe.mths_since_last_delinq.isnull()  ,('mths_since_last_delinq')] = '0'
dataframe.loc[dataframe.mths_since_last_record.isnull()  ,('mths_since_last_record')] = '0'
dataframe.loc[dataframe.initial_list_status == 'f'  ,('initial_list_status')] = '1'
dataframe.loc[dataframe.initial_list_status == 'w'  ,('initial_list_status')] = '0'
dataframe.loc[dataframe.application_type == 'INDIVIDUAL'  ,('application_type')] = '1'
dataframe.loc[dataframe.application_type == 'JOINT'  ,('application_type')] = '0'
dataframe.loc[:,('last_week_pay')] = dataframe['last_week_pay'].str.split('th week').str[0]


dataframe

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,grade,emp_length,home_ownership,annual_inc,verification_status,...,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status,nstatus
0,14350,14350,14350.0,36,19.19,5,9,1,28700.0,1,...,0.0,74,1,26,0.0,0.0,28699.0,30800.0,0,1
1,4800,4800,4800.0,36,10.99,2,0,3,65000.0,1,...,0.0,0,1,9,0.0,0.0,9974.0,32900.0,0,1
2,10000,10000,10000.0,36,7.26,1,2,1,45000.0,3,...,0.0,0,1,9,0.0,65.0,38295.0,34900.0,0,1
3,15000,15000,15000.0,36,19.72,4,11,2,105000.0,3,...,0.0,0,1,135,0.0,0.0,55564.0,24700.0,0,1
4,16000,16000,16000.0,36,10.64,2,11,2,52000.0,2,...,0.0,0,1,96,0.0,0.0,47159.0,47033.0,0,1
5,15000,15000,14950.0,36,8.90,1,2,3,120000.0,3,...,0.0,0,1,113,0.0,0.0,350619.0,29500.0,0,1
6,5000,5000,4975.0,36,7.90,1,5,2,75000.0,1,...,0.0,0,1,117,0.0,1023.0,13272.0,55500.0,1,0
7,6000,6000,6000.0,36,9.17,2,8,3,54000.0,3,...,0.0,54,1,78,0.0,0.0,272579.0,11800.0,0,1
8,6000,6000,6000.0,36,13.99,3,7,3,92000.0,3,...,0.0,0,1,44,0.0,0.0,281521.0,62100.0,0,1
9,34550,34550,34550.0,60,17.14,4,2,3,72000.0,2,...,0.0,0,1,52,0.0,0.0,76034.0,33200.0,0,1


In [5]:
inputX = dataframe.ix[:,:34].as_matrix()
inputY = dataframe.loc[:,['loan_status','nstatus']].as_matrix()

inputX


array([[14350, 14350, 14350.0, '36 ', 19.19, '5', '9 ', '1', 28700.0, '1',
        '1', '3', 0.0, 1.0, 50.0, 75.0, 14.0, 1.0, 22515.0, 73.1, 28.0,
        '1', 1173.84, 0.0, 0.0, 0.0, 0.0, 74.0, '1', '26', 0.0, 0.0,
        28699.0, 30800.0],
       [4800, 4800, 4800.0, '36 ', 10.99, '2', '0', '3', 65000.0, '1', '1',
        '2', 0.0, 1.0, '0', '0', 6.0, 0.0, 7624.0, 23.2, 13.0, '0', 83.95,
        0.0, 0.0, 0.0, 0.0, '0', '1', '9', 0.0, 0.0, 9974.0, 32900.0],
       [10000, 10000, 10000.0, '36 ', 7.26, '1', '2 ', '1', 45000.0, '3',
        '1', '3', 0.0, 0.0, '0', '0', 5.0, 0.0, 10877.0, 31.2, 19.0, '0',
        56.47, 0.0, 0.0, 0.0, 0.0, '0', '1', '9', 0.0, 65.0, 38295.0,
        34900.0],
       [15000, 15000, 15000.0, '36 ', 19.72, '4', '11', '2', 105000.0, '3',
        '1', '3', 0.0, 2.0, 46.0, '0', 10.0, 0.0, 13712.0, 55.5, 21.0, '1',
        4858.62, 0.0, 0.0, 0.0, 0.0, '0', '1', '135', 0.0, 0.0, 55564.0,
        24700.0],
       [16000, 16000, 16000.0, '36 ', 10.64, '2', '11', 

In [6]:
inputY


array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1]])

In [7]:
learning_rate = 0.000001
training_epochs = 4000
display_steps = 50
n_sample = inputY.size


In [8]:
x = tf.placeholder(tf.float32,[None,34])
W = tf.Variable(tf.zeros([34,2]))
b = tf.Variable(tf.zeros([2]))
y_values = tf.add(tf.matmul(x,W),b)
y = tf.nn.softmax(y_values)
y_ = tf.placeholder(tf.float32,[None,2])
cost = tf.reduce_sum(tf.pow(y_ - y, 2)) / (2*n_sample)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)


In [9]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [10]:
for i in range(training_epochs):
    sess.run(optimizer,feed_dict = {x:inputX,y_:inputY})
    
    if (i) % display_steps == 0:
        cc = sess.run(cost, feed_dict = {x:inputX,y_:inputY})
        print "training step:", '%04d' % (i), "cost=","{:.9f}".format(cc)
        
print "optimization finished!"
training_cost = sess.run(cost,feed_dict={x: inputX,y_:inputY})
print "training cost=",training_cost, "W=", sess.run(W), "b=", sess.run(b),'\n'

training step: 0000 cost= 0.050000001
training step: 0050 cost= 0.050000001
training step: 0100 cost= 0.050000001
training step: 0150 cost= 0.050000001
training step: 0200 cost= 0.050000001
training step: 0250 cost= 0.050000001
training step: 0300 cost= 0.050000001
training step: 0350 cost= 0.050000001
training step: 0400 cost= 0.050000001
training step: 0450 cost= 0.050000001
training step: 0500 cost= 0.050000001
training step: 0550 cost= 0.050000001
training step: 0600 cost= 0.050000001
training step: 0650 cost= 0.050000001
training step: 0700 cost= 0.050000001
training step: 0750 cost= 0.050000001
training step: 0800 cost= 0.050000001
training step: 0850 cost= 0.050000001
training step: 0900 cost= 0.050000001
training step: 0950 cost= 0.050000001
training step: 1000 cost= 0.050000001
training step: 1050 cost= 0.050000001
training step: 1100 cost= 0.050000001
training step: 1150 cost= 0.050000001
training step: 1200 cost= 0.050000001
training step: 1250 cost= 0.050000001
training ste

In [11]:
sess.run(y, feed_dict = {x:inputX})


array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]], dtype=float32)